# How to start

Before starting you must:
- Ensure that `scipp` and `mantid` are on your `PYTHONPATH`.
- Generate the `config.py` file using `make_config.py`. Refer to the `README.md` or `python make_config.py --help` for information.
- Install dependencies : `conda install fabio tifffile` (used for image handling)

For `scipp` and `mantid` follow instructions at: https://scipp.readthedocs.io/en/latest/getting-started/installation.html.

Converted to use scipp and notebook from [this file](https://git.esss.dk/testbeamline/gp2/blob/1c69213b1124982bbbe762da9c6c6457a49f2a92/reduce.py) by Dimitar Tasev on 2020-01-13

In [ ]:
try:
    import scipp
except ImportError as e:
    print("scipp is not available in the PYTHONPATH")
    raise e
    
try:
    import mantid
except ImportError as e:
    print("mantid is not available in the PYTHONPATH")
    raise e
    
try:
    import scippconfig
except ImportError as e:
    print("scippconfig is not available. Make sure you have generated it with `make_config.py`.")
    raise e

In [ ]:
# Lets get everything set up
import os

import scipp as sc
import numpy as np
from scipp import Dim
from mantid.simpleapi import SaveNexusESS

import imaging

DATA_DIR_NAME = "data_GP2"
experiment_dir = scippconfig.script_root
data_dir = os.path.join(scippconfig.script_root, DATA_DIR_NAME)

if not os.path.exists(data_dir):
    raise FileNotFoundError("The following data directory does not exist,"
                            f" check your make_config.py:\n{data_dir}")

In [ ]:
# Customisable Options:

# Whether or not to do the plotting.
do_plots = True

# defining grouping of 2D detector pixels
grouping_number = 3
nx_target = grouping_number
ny_target = grouping_number

# Rebin regions for each of the 5 frames
# in the format of [bin-start, bin-end, bin width].
# used to crop each image, before stitching them together
frame_parameters = [(15167, 23563, 64),
                    (24393, 32758, 64),
                    (33365, 40708, 64),
                    (41410, 48019, 64),
                    (49041, 55311, 64),
                    (56077, 59872, 64)]

# Used to shift the cropped frames so that their bins overlap 
# before summing them together into a single frame
frame_shift_increments = [-6630.0, -2420.0, -2253.0, -2095.0, -1946.0, -1810.0]
frame_shifts = [sum(frame_shift_increments[:i + 1]) for i in range(len(frame_shift_increments))]

# Used to rebin the summed frame in order to
# cut off frames that contain no data
rebin_parameters = {"start": 8500, "stop": 43000, "width": 64}

In [ ]:
# let's get the process started:
tofs_path = os.path.join(data_dir, 'metadata', 'GP2_BCC_time_values.txt')
sample_path = os.path.join(data_dir, 'Timeslices WFM BBC Steel')
ob_path = os.path.join(data_dir, 'Timeslices WFM Open Beam')
instrument_file = os.path.join(experiment_dir, 'IDF', 'V20_Definition_GP2.xml')

ds = sc.Dataset()

# Load X values from the TOF file
ds.coords[Dim.Tof] = sc.Variable([Dim.Tof], unit=sc.units.us, values=imaging.read_x_values(tofs_path))
ds.coords[Dim.Tof] *= 1e3

In [ ]:
# Load the data into the dataset
ds["sample"] = imaging.tiffs_to_variable(sample_path)
ds["reference"] = imaging.tiffs_to_variable(ob_path)

sc.plot.plot(sc.sum(ds, Dim.Spectrum)) if do_plots else None

In [ ]:
# Adds a coordinate for the spectra
ds.coords[Dim.Spectrum] = sc.Variable([Dim.Spectrum], values=np.arange(ds["sample"].shape[1]))
ds

In [ ]:
stitched = sc.Dataset(coords={Dim.Tof: sc.Variable([Dim.Tof],
                      values=np.arange(start=rebin_parameters["start"], stop=rebin_parameters["stop"],
                                       step=rebin_parameters["width"], dtype=np.float64),
                      unit=sc.units.us)})

stitched["sample"] = imaging.stitch(ds["sample"], frame_parameters, frame_shifts, rebin_parameters)
stitched["reference"] = imaging.stitch(ds["reference"], frame_parameters, frame_shifts, rebin_parameters)

sc.plot.plot(sc.sum(stitched, Dim.Spectrum)) if do_plots else None

In [ ]:
stitched["normalized"] = stitched["sample"] / stitched["reference"]
replacement=sc.Variable(value=0.0, variance=0.0)
kwargs = {"nan" : replacement, "posinf" : replacement, "neginf" : replacement}
sc.nan_to_num(stitched["normalized"].data, out=stitched["normalized"].data, **kwargs)

In [ ]:
stitched.coords["detector_mapping"] = imaging.make_detector_groups(324, 324, nx_target, ny_target)
stitched.coords["detector_mapping"]

In [ ]:
dm1d = sc.groupby(stitched["normalized"], "detector_mapping")
grouped = sc.Dataset()
grouped["normalized_grpd"] = dm1d.sum(Dim.Spectrum)

In [ ]:
# Plot the first group
grouped
sc.plot.plot(grouped["normalized_grpd"]['detector_mapping', 0]) if do_plots else None

In [ ]:
# Adds the component info needed for converting units
sc.compat.mantid.load_component_info(stitched, instrument_file)

grouped.coords["source_position"] = stitched.coords["source_position"]
grouped.coords["sample_position"] = stitched.coords["sample_position"]

In [ ]:
# Note: this cell is currently a workaround until 
# https://github.com/scipp/scipp/issues/823 is done.
# Then it should be possible to group the positions label directly.

# makes the position a DataConstProxy otherwise groupby won't take it
stitched["position"] = stitched.coords["position"]
dm1d = sc.groupby(stitched["position"], "detector_mapping")
position = dm1d.mean(Dim.Spectrum)

# can't do stitched.labels["position"] = position because Labels won't take a DataArray
# also can't do stitched.labels["position"] = sc.Variable(position) because then sc.convert 
# thinks stitched.labels["position"] is dimensionless (as it's actually still a DataArray)

pos = np.zeros((position.shape[0], 3))
for i, val in enumerate(position.values):
    pos[i, :] = val

# finally add it back!
grouped.coords["position"] = sc.Variable(position.dims, pos, unit=sc.units.m, dtype=position.dtype)


In [ ]:
grouped["normalized_wl"] = sc.neutron.convert(grouped["normalized_grpd"], Dim.Tof, Dim.Wavelength)
grouped

In [ ]:
sc.plot.plot(grouped["normalized_wl"]["detector_mapping", 0]) if do_plots else None

In [ ]:
x_coords = grouped["normalized_wl"].coords["wavelength"]
x_dim = grouped["normalized_wl"].dims[0]
x = x_coords.values

# Mantid expects the data in a different shape
# which is spectrum as outer-most dimension.
y = np.transpose(grouped["normalized_wl"].values)
e = np.transpose(grouped["normalized_wl"].variances)

ws = sc.compat.mantid.to_workspace_2d(x, y, e, str(x_dim), instrument_file)
ws

In [ ]:
# If you want to save the workspace uncomment this line
SaveNexusESS(ws, "scipp_normalized_wl.nxs")

In [ ]:
fit_ds = sc.compat.mantid.fit(ws, 
                    function='name=LinearBackground,A0=5000,A1=0;name=UserFunction,Formula=h*erfc(a*(x-x0)),h=5000,a=-0.5,x0=4',
                    workspace_index=2, start_x=3.6, end_x=4.4)

In [ ]:
plot_handle = sc.plot.plot(fit_ds["workspace"].value)
plot_handle = sc.plot.plot(fit_ds["workspace"].value, collapse=Dim.Wavelength)